In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from seaborn import load_dataset
df = sns.load_dataset("titanic")

In [2]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [3]:
df.isnull().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [15]:
data = df.copy()
data["age"].fillna(df["age"].median(skipna=True), inplace=True)
data["embarked"].fillna(df['embarked'].value_counts().idxmax(), inplace=True)
data.drop('deck', axis=1, inplace=True)
data.drop('embark_town', axis=1, inplace=True)
data.drop('who', axis=1, inplace=True)

In [16]:
data.isnull().sum()

survived      0
pclass        0
sex           0
age           0
sibsp         0
parch         0
fare          0
embarked      0
class         0
adult_male    0
alive         0
alone         0
dtype: int64

In [17]:
data.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,adult_male,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,True,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,False,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,False,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,False,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,True,no,True


In [21]:
data['TravelAlone'] = np.where((data["sibsp"]+data["parch"])>0, 0, 1)
data.drop('sibsp', axis=1, inplace=True)
data.drop('parch', axis=1, inplace=True)

In [24]:
training = pd.get_dummies(data, columns=["pclass","embarked","sex"])
training.drop('sex_female', axis=1, inplace=True)
training.drop('class', axis=1, inplace=True)
training.drop('adult_male', axis=1, inplace=True)
training.drop('alive', axis=1, inplace=True)
training.drop('alone', axis=1, inplace=True)

final_train = training
final_train.head()

,survived,age,fare,TravelAlone,pclass_1,pclass_2,pclass_3,embarked_C,embarked_Q,embarked_S,sex_male
0,0,22.0,7.2500,0,0,0,1,0,0,1,1
1,1,38.0,71.2833,0,1,0,0,1,0,0,0
2,1,26.0,7.9250,1,0,0,1,0,0,1,0
3,1,35.0,53.1000,0,1,0,0,0,0,1,0
4,0,35.0,8.0500,1,0,0,1,0,0,1,1


In [31]:
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model, metrics
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression

cols = ["age","fare","TravelAlone","pclass_1","pclass_2","embarked_C","embarked_S","sex_male"] 
X = final_train[cols]
y = final_train['survived']


In [32]:
X.shape

(891, 8)

In [33]:
y.shape

(891,)

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)


/Users/wintwinthtae/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [35]:
accuracy_score(y_pred, y_test)

0.7877094972067039

In [36]:
print(confusion_matrix(y_pred, y_test))

[[88 21]
 [17 53]]


In [37]:
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.84      0.81      0.82       109
           1       0.72      0.76      0.74        70

    accuracy                           0.79       179
   macro avg       0.78      0.78      0.78       179
weighted avg       0.79      0.79      0.79       179



In [39]:
result =pd.DataFrame({
    "Actual":y_train,
    "Predict":logreg.predict(X_train)
})

result.to_csv("result.csv", index=False)

result.tail()

,Actual,Predict
106,1,1
270,0,0
860,0,0
435,1,1
102,0,0
